## Beta distributions for CTR regularization



In [1]:
import pandas as pd
import numpy as np
import plotnine as p9
import statistics as stats

### Some example data

This data has impressions and clicks, which we will convert to `ctr`.

In [2]:
# read-in the downloaded data
df = pd.read_csv("click_log.csv")
df

,resource_id,term,impressions,downloads
0,40,background,3,1
1,40,glass,7,1
2,40,guitar,8,1
3,40,metal texture,9,1
4,40,patterns,9,1
...,...,...,...,...
1048570,237224,pattern background,4,1
1048571,237224,patterns,3,1
1048572,237224,school,3,1
1048573,237224,scribble,10,1


In [ ]:
# create a new column for CTR
df['ctr'] = df['___'] / df['____']

In [ ]:
# calculate the average CTR as a variable
mean_ctr = stats._____(df['____'])

In [ ]:
# calculate the variance CTR
var_ctr = stats._____(df['____'])

In [ ]:
# calculate sum(alpha, beta)
alpha_beta = mean_ctr * (1 - mean_ctr)  / var_ctr - 1
alpha_beta

In [ ]:
# calculate alpha
alpha = mean_ctr * alpha_beta
alpha

In [ ]:
# calculate beta
beta = (1-mean_ctr) * alpha_beta
beta

In [ ]:
# calulate expected CTR
expected_ctr = alpha / (alpha + beta)
expected_ctr

### Focus on the query 'duck'

To show how this can adjust our CTR metric let's look at for just a single query and handful of documents. You should notice that the adjusted values are all closer to the global average that the were as raw CTR values. The magnitude of this shift is based on the number of observations

The concept is shrinking individual resource CTRs towards to the global average CTR. The less data we have on a given resource the more we trust the global average. The more data we have for a resource the more we trust it.

This process gives a robust estimates when we only have a few observations, which is often the case for search.


In [ ]:
ducks = df.loc[df['term'] == 'duck']
ducks = ducks.head(100)
ducks

In [ ]:
# caculate adjusted CTR
ducks['adjusted_ctr'] = (alpha + ducks['clicks']) / (alpha + beta + ducks['impressions'])
ducks

In [ ]:
# visualize it to see the shift

ducks_long = pd.melt(ducks[['resource_id', 'ctr', 'adjusted_ctr', 'impressions']], id_vars=['resource_id', 'impressions'], value_vars=['ctr', 'adjusted_ctr'])

{
    p9.ggplot(ducks_long, p9.aes('value', 'resource_id', color = 'variable', size = 'impressions')) +
    p9.geom_vline(xintercept = expected_ctr, linetype='dashed') +
    p9.geom_point(alpha = .5) +
    p9.scale_x_log10()
}

### A small salient example of shrinkage rate

See this at query scale can be over-whelming. This sections is about seeing what happens with only a few data points. You are encouraged to change the clicks and impressions values.

In [ ]:
df2 = pd.DataFrame(
    data = {
        'id': [1,2,3],
        'clicks': [1, 1, 1],
        'impressions': [2, 9, 99]
    }
)

In [ ]:
df2['ctr_raw'] = ____

In [ ]:
df2['ctr_adjusted'] = ____

In [ ]:
df2 = pd.melt(df2[['id', 'ctr_raw', 'ctr_adjusted']], id_vars='id', value_vars=['ctr_raw', 'ctr_adjusted'])

In [ ]:
df2

In [ ]:
{
    p9.ggplot(df2, p9.aes(_____, _____ color = 'variable')) +
    p9.geom_vline(xintercept = _______, linetype='dashed') + # global expected CTR
    p9.geom_point() +
    p9.scale_x_log10()
}